In [1]:
import os
os.environ['OPENTSDB_PYTHON_METRICS_TEST_MODE'] = 'True'

In [2]:
from banzai.calibrations import make_master_calibrations
import requests
from banzai_nres import settings
from banzai import dbs
from banzai.utils.stage_utils import run_pipeline_stages
import logging
from banzai.logs import set_log_level
from glob import glob

import pkg_resources

Setup logging and some settings show we know where the DB should be.
------

In [3]:
set_log_level('DEBUG')
logger = logging.getLogger('banzai')

In [4]:
from banzai import main

2020-06-23 15:54:18.738     INFO: <ipython-input-7-fc7e203332d2>: adding bpm test_data/lsc/nres01/bpm/bpm-lsc-nres01-fl09-20180215.fits.fz to the database | {"processName": "MainProcess"}
2020-06-23 15:55:04.771     INFO: <ipython-input-8-f0a511cf5bd8>: adding bpm test_data/lsc/nres01/bpm/bpm-lsc-nres01-fl09-20180215.fits.fz to the database | {"processName": "MainProcess"}
2020-06-23 15:55:28.292     INFO:          stages: Running banzai.stages.BadPixelMaskLoader | {"filename": "lscnrs01-fl09-20180313-0044-b00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "BIAS", "filter": null, "processName": "MainProcess"}
2020-06-23 15:55:28.587     INFO:    calibrations: Applying master calibration | {"filename": "lscnrs01-fl09-20180313-0044-b00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "BIAS", "filter": null, "master_calibration": "bpm-lsc-nres01-fl09-20180215.fits.

2020-06-23 15:55:36.363     INFO:            trim: Trimming image | {"filename": "lscnrs01-fl09-20180313-0045-b00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "BIAS", "filter": null, "processName": "MainProcess"}
2020-06-23 15:55:36.774     INFO:          stages: Running banzai.stages.BiasMasterLevelSubtractor | {"filename": "lscnrs01-fl09-20180313-0045-b00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "BIAS", "filter": null, "processName": "MainProcess"}
2020-06-23 15:55:37.415     INFO:          stages: Running banzai.stages.BiasComparer | {"filename": "lscnrs01-fl09-20180313-0045-b00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "BIAS", "filter": null, "processName": "MainProcess"}
2020-06-23 15:55:37.422    ERROR:    calibrations: No master bias to compare to, Flagging image as bad. | {"fi

2020-06-23 15:55:58.729     INFO:          stages: Running banzai.stages.BadPixelMaskLoader | {"filename": "lscnrs01-fl09-20180313-0048-d00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DARK", "filter": null, "processName": "MainProcess"}
2020-06-23 15:55:58.990     INFO:    calibrations: Applying master calibration | {"filename": "lscnrs01-fl09-20180313-0048-d00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DARK", "filter": null, "master_calibration": "bpm-lsc-nres01-fl09-20180215.fits.fz", "processName": "MainProcess"}
2020-06-23 15:55:59.029     INFO:          stages: Running banzai.stages.OverscanSubtractor | {"filename": "lscnrs01-fl09-20180313-0048-d00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DARK", "filter": null, "processName": "MainProcess"}
2020-06-23 15:55:59.061     INFO:   

2020-06-23 15:56:05.319    ERROR:    calibrations: No master dark to compare to, Flagging image as bad. | {"filename": "lscnrs01-fl09-20180313-0047-d00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DARK", "filter": null, "processName": "MainProcess"}
2020-06-23 15:56:08.356     INFO:          stages: Running banzai.stages.BadPixelMaskLoader | {"filename": "lscnrs01-fl09-20180313-0049-d00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DARK", "filter": null, "processName": "MainProcess"}
2020-06-23 15:56:08.651     INFO:    calibrations: Applying master calibration | {"filename": "lscnrs01-fl09-20180313-0049-d00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DARK", "filter": null, "master_calibration": "bpm-lsc-nres01-fl09-20180215.fits.fz", "processName": "MainProcess"}
2020-06-23 15:56:08.694 

2020-06-23 15:56:25.443     INFO:          stages: Running banzai.stages.PoissonInitializer | {"filename": "lscnrs01-fl09-20180313-0013-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-06-23 15:56:25.635     INFO:          stages: Running banzai.stages.DarkSubtractor | {"filename": "lscnrs01-fl09-20180313-0013-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-06-23 15:56:26.436     INFO:    calibrations: Applying master calibration | {"filename": "lscnrs01-fl09-20180313-0013-w00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "master_calibration": "lscnrs01-fl09-20180313-dark-bin1x1.fits.fz", "processName": "MainProcess"}
2020-06-23 15:56:30.00

...

2020-06-23 16:02:17.759     INFO:          traces: Fitting smooth spline to order 130 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-06-23 16:02:18.142     INFO:          traces: Fitting smooth spline to order 131 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-06-23 16:02:18.567     INFO:          traces: Fitting smooth spline to order 132 | {"filename": "lscnrs01-fl09-20180313-lampflat-bin1x1-110.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "LAMPFLAT", "filter": null, "processName": "MainProcess"}
2020-06-23 16:02:19.028     INFO:          traces: Fitting smooth s

2020-06-23 16:02:57.822     INFO:          stages: Running banzai.stages.GainNormalizer | {"filename": "lscnrs01-fl09-20180313-0008-a00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DOUBLE", "filter": null, "processName": "MainProcess"}
2020-06-23 16:02:57.823     INFO:            gain: Multiplying by gain | {"filename": "lscnrs01-fl09-20180313-0008-a00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DOUBLE", "filter": null, "processName": "MainProcess"}
2020-06-23 16:02:57.862     INFO:          stages: Running banzai.stages.Trimmer | {"filename": "lscnrs01-fl09-20180313-0008-a00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DOUBLE", "filter": null, "processName": "MainProcess"}
2020-06-23 16:02:57.863     INFO:            trim: Trimming image | {"filename": "lscnrs01-fl09-20180313-0008-a00.f

2020-06-23 16:03:09.174     INFO:            trim: Trimming image | {"filename": "lscnrs01-fl09-20180313-0020-a00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DOUBLE", "filter": null, "processName": "MainProcess"}
2020-06-23 16:03:09.458     INFO:          stages: Running banzai.stages.BiasSubtractor | {"filename": "lscnrs01-fl09-20180313-0020-a00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DOUBLE", "filter": null, "processName": "MainProcess"}
2020-06-23 16:03:10.156     INFO:    calibrations: Applying master calibration | {"filename": "lscnrs01-fl09-20180313-0020-a00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DOUBLE", "filter": null, "master_calibration": "lscnrs01-fl09-20180313-bias-bin1x1.fits.fz", "processName": "MainProcess"}
2020-06-23 16:03:10.376     INFO:          stages: Run

2020-06-23 16:03:22.914     INFO:          stages: Running banzai.stages.PoissonInitializer | {"filename": "lscnrs01-fl09-20180313-0022-a00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DOUBLE", "filter": null, "processName": "MainProcess"}
2020-06-23 16:03:23.040     INFO:          stages: Running banzai.stages.DarkSubtractor | {"filename": "lscnrs01-fl09-20180313-0022-a00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DOUBLE", "filter": null, "processName": "MainProcess"}
2020-06-23 16:03:23.692     INFO:    calibrations: Applying master calibration | {"filename": "lscnrs01-fl09-20180313-0022-a00.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DOUBLE", "filter": null, "master_calibration": "lscnrs01-fl09-20180313-dark-bin1x1.fits.fz", "processName": "MainProcess"}
2020-06-23 16:03:26.915     

2020-06-23 16:03:37.477     INFO:    calibrations: Making master frames | {"type": "1m0-NRES-SciCam", "site": "lsc", "camera": "fl09", "obstype": "DOUBLE", "min_date": "2017-01-01", "max_date": "2019-01-01", "processName": "MainProcess"}
2020-06-23 16:03:43.712     INFO:          stages: Running banzai.stages.ArcStacker | {"filename": "lscnrs01-fl09-20180313-0008-a91.fits.fz", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DOUBLE", "filter": null, "processName": "MainProcess"}
2020-06-23 16:03:47.488     INFO:    calibrations: Created master calibration stack | {"filename": "lscnrs01-fl09-20180313-double-bin1x1-011.fits", "site": "lsc", "instrument": "nres01", "epoch": "20180313", "request_num": "UNSPECIFIED", "obstype": "DOUBLE", "filter": null, "calibration_type": "DOUBLE", "processName": "MainProcess"}
2020-06-23 16:03:47.494     INFO:          stages: Running banzai.stages.ArcStacker | {"filename": "lscnrs01-fl09-20180313-0023-

In [5]:
os.environ['DB_ADDRESS'] = 'sqlite:///test_data/test.db'

settings.processed_path= os.path.join(os.getcwd(), 'test_data')
settings.fpack=True
settings.db_address = os.environ['DB_ADDRESS']

Download some test data from the archive
----------------------------------------

In [6]:
# make directories for the test dataset.
raw_data_dir = 'test_data/lsc/nres01/20180313/raw'
os.makedirs(raw_data_dir, exist_ok=True)
bpm_dir = 'test_data/lsc/nres01/bpm/'
os.makedirs(bpm_dir, exist_ok=True)

In [7]:
# choose which files we will download for our test dataset.
bpm_filename = 'bpm-lsc-nres01-fl09-20180215.fits.fz'
test_filenames = ['lscnrs01-fl09-20180313-0001-w00.fits.fz', 'lscnrs01-fl09-20180313-0002-w00.fits.fz',
                  'lscnrs01-fl09-20180313-0003-w00.fits.fz', 'lscnrs01-fl09-20180313-0004-w00.fits.fz',
                  'lscnrs01-fl09-20180313-0005-w00.fits.fz', 'lscnrs01-fl09-20180313-0006-a00.fits.fz',
                  'lscnrs01-fl09-20180313-0007-a00.fits.fz', 'lscnrs01-fl09-20180313-0008-a00.fits.fz',
                  'lscnrs01-fl09-20180313-0009-w00.fits.fz', 'lscnrs01-fl09-20180313-0010-w00.fits.fz',
                  'lscnrs01-fl09-20180313-0011-w00.fits.fz', 'lscnrs01-fl09-20180313-0012-w00.fits.fz',
                  'lscnrs01-fl09-20180313-0013-w00.fits.fz', 'lscnrs01-fl09-20180313-0014-w00.fits.fz',
                  'lscnrs01-fl09-20180313-0015-w00.fits.fz', 'lscnrs01-fl09-20180313-0016-w00.fits.fz',
                  'lscnrs01-fl09-20180313-0017-w00.fits.fz', 'lscnrs01-fl09-20180313-0018-w00.fits.fz',
                  'lscnrs01-fl09-20180313-0019-a00.fits.fz', 'lscnrs01-fl09-20180313-0020-a00.fits.fz',
                  'lscnrs01-fl09-20180313-0021-a00.fits.fz', 'lscnrs01-fl09-20180313-0022-a00.fits.fz',
                  'lscnrs01-fl09-20180313-0023-a00.fits.fz', 'lscnrs01-fl09-20180313-0042-b00.fits.fz',
                  'lscnrs01-fl09-20180313-0043-b00.fits.fz', 'lscnrs01-fl09-20180313-0044-b00.fits.fz',
                  'lscnrs01-fl09-20180313-0045-b00.fits.fz', 'lscnrs01-fl09-20180313-0046-b00.fits.fz',
                  'lscnrs01-fl09-20180313-0047-d00.fits.fz', 'lscnrs01-fl09-20180313-0048-d00.fits.fz',
                  'lscnrs01-fl09-20180313-0049-d00.fits.fz', 'lscnrs01-fl09-20180313-0028-e00.fits.fz',
                  'lscnrs01-fl09-20180313-0029-e00.fits.fz', 'lscnrs01-fl09-20180313-0030-e00.fits.fz',
                  'lscnrs01-fl09-20180313-0031-e00.fits.fz']

bpm_frame_id = '22529799'
test_frame_ids = ['8148793', '8148822', '8148805', '8148824', '8148826', '8148876', '8148898',
                  '8148932', '8148960', '8148978', '8149024', '8149104', '8149068', '8149090',
                  '8149128', '8149173', '8149217', '8151252', '8149314', '8149388', '8149469',
                  '8149547', '8149570', '8156341', '8156360', '8156366', '8156376', '8156385',
                  '8156433', '8156500', '8156560', '8151997', '8152050', '8152110', '8152153']

In [8]:
# THIS DOWNLOADS A NEW DATA SET. DO NOT RUN IF YOU HAVE ALREADY DOWNLOADED THE DATA
# Copy the data into the correct directory
for test_filename, frame_id in zip(test_filenames, test_frame_ids):
    archive_url = f'https://archive-api.lco.global/frames/{frame_id}'
    frame_info = requests.get(archive_url).json()
    with open(os.path.join(raw_data_dir, test_filename), 'wb') as f:
        f.write(requests.get(frame_info['url']).content)
        
archive_url = f'https://archive-api.lco.global/frames/{bpm_frame_id}'
frame_info = requests.get(archive_url).json()
with open(os.path.join(bpm_dir, bpm_filename), 'wb') as f:
    f.write(requests.get(frame_info['url']).content)

Initialize the db and context object for the rest of the pipeline
-----------------------------------------------------------------

In [6]:
# set up the context object.
import banzai
context = banzai.main.parse_args(settings, parse_system_args=False)

In [8]:
# THIS MAKES A FRESH DATABASE. DO NOT RUN ME UNLESS YOU WANT TO DELETE YOUR test.db AND MAKE A FRESH ONE
os.system(f'banzai_create_db --db-address={os.environ["DB_ADDRESS"]}')
os.system(f'banzai_add_site --site elp --latitude 30.67986944 --longitude -104.015175 --elevation 2027 --timezone -6 --db-address={os.environ["DB_ADDRESS"]}')
os.system(f'banzai_add_site --site lsc --latitude -30.1673833333 --longitude -70.8047888889 --elevation 2198 --timezone -4 --db-address={os.environ["DB_ADDRESS"]}')   
os.system(f'banzai_add_instrument --site lsc --camera fl09 --name nres01 --camera-type 1m0-NRES-SciCam --db-address={os.environ["DB_ADDRESS"]}')
os.system(f'banzai_add_instrument --site elp --camera fl17 --name nres02 --camera-type 1m0-NRES-SciCam --db-address={os.environ["DB_ADDRESS"]}')
for bpm_filename in glob('test_data/*/nres??/bpm/*bpm*'):
    logger.info(f'adding bpm {bpm_filename} to the database')
    os.system(f'banzai_nres_add_bpm --filename {bpm_filename} --db-address={os.environ["DB_ADDRESS"]}')

Process and Stack Bias Files
----------------------------

In [9]:
bias_files = glob(os.path.join('test_data/*/nres??/*/raw/*b00*'))
for bias_file in bias_files: 
    run_pipeline_stages([{'path': bias_file}], context) 

In [10]:
def mark_frames_as_good(filenames):
    for filename in glob(f'test_data/*/nres??/*/processed/{filenames}'):
        dbs.mark_frame(os.path.basename(filename), "good", db_address=os.environ['DB_ADDRESS'])

In [11]:
mark_frames_as_good('*b91*')

In [12]:
instrument = dbs.get_instruments_at_site('lsc', settings.db_address)[0]

In [13]:
make_master_calibrations(instrument, 'BIAS', '2017-01-01', '2019-01-01', context) 

Process and Stack Dark Files
----------------------------

In [14]:
dark_files = glob('test_data/*/nres??/*/raw/*d00*')
for dark_file in dark_files: 
    run_pipeline_stages([{'path': dark_file}], context)

In [15]:
mark_frames_as_good('*d91*')

In [16]:
make_master_calibrations(instrument, 'DARK', '2017-01-01', '2019-01-01', context) 

Process and stack lamp flats. This is where we also solve for the traces and profile.
----------

In [17]:
flat_files = glob('test_data/*/nres??/*/raw/*w00*')
for flat_file in flat_files: 
    run_pipeline_stages([{'path': flat_file}], context)

In [18]:
make_master_calibrations(instrument, 'LAMPFLAT', '2017-01-01', '2019-01-01', context)

Process and stack arc lamps. Find the wavelength solution
--------

In [19]:
arc_files = glob('test_data/*/nres??/*/raw/*a00*')
for arc_file in arc_files: 
    run_pipeline_stages([{'path': arc_file}], context)

In [ ]:
make_master_calibrations(instrument, 'DOUBLE', '2017-01-01', '2019-01-01', context)

Process the science spectra
------

In [ ]:
science_files = glob('test_data/*/nres??/*/raw/*e00*')
for science_file in science_files: 
    run_pipeline_stages([{'path': science_file}], context)